In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect


NOTE: User is responsible for checking the content of datasets and the applicable licenses and determining if suitable for the intended use.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'r1.22.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

# Update numba and restart (this is required to update internal numba version of Colab)

# In a conda environment, you would use the following command
# Update Numba to > 0.54
# conda install -c conda-forge numba>=0.54
# or
# conda update -c conda-forge numba>=0.54

# For pip based environments,
# Update Numba to > 0.54
import os
import signal

!pip install --upgrade numba

# This will kill the kernel, click next cell to import the latest numba
os.kill(os.getpid(), signal.SIGKILL)

# Buffered Transducer evaluation with Longest Common Subsequence Merge

In the [Buffered Transducer Inference](https://github.com/NVIDIA/NeMo/blob/stable/tutorials/asr/Buffered_Transducer_Inference.ipynb) tutorial, we discussed how we could perform Streaming/Buffered inference with Transducer models by using a technique which we term as `"Middle Token" selection` from a buffer.

In this notebook, we will perform buffered ASR speech recognition and utilize another algorithm to merge buffers during inference. We term this method as the `"Longest Common Subsequence" (LCS) Merge` algorithm.

While the `Middle Token` algorithm works well in general, it is not a perfect merge algorithm and can make mistakes. We, therefore, compare against the `LCS Merge` algorithm and discuss the merits of both approaches. 

-----

You may use this script [ASR Chunked Streaming Inference](https://github.com/NVIDIA/NeMo/blob/stable/examples/asr/asr_chunked_inference/rnnt/speech_to_text_buffered_infer_rnnt.py) to transcribe long audio files with Transducer models as well as experiment with both merge algorithms. 


------

**Note**: It is highly recommended to review the ``Streaming ASR`` tutorial for a good overview of how streaming/buffered inference works for CTC models and the underlying motivation of streaming ASR itself.

------

# Prepare the dataset

We will reuse the Librispeech dev-clean subset of [Mini Librispeech](https://www.openslr.org/31/). This time, we will not concatenate the audio segments but simply evaluate them in buffered mode over all the audio samples.

**Note**: Conformer inference over the entire dev set will take an exorbitant amount of time on the CPU. We recommend the use of GPU for this tutorial.

## Download and prepare Mini Librispeech

In [ ]:
#@title Prepare dataset and manifest for Libripeech Dev Clean subset.
import os

if not os.path.exists("scripts/get_librispeech_data.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/stable/scripts/dataset_processing/get_librispeech_data.py

# If something goes wrong during data processing, un-comment the following line to delete the cached dataset 
# !rm -rf datasets/mini-dev-clean
!mkdir -p datasets/mini-dev-clean

!python scripts/get_librispeech_data.py \
  --data_root "datasets/mini-dev-clean/" \
  --data_sets dev_clean_2 \
  --num_workers=10 \
  --log

In [ ]:
manifest = os.path.join(os.getcwd(), "datasets/mini-dev-clean/dev_clean_2.json")
print("Manifest path :", manifest)

# Prepare the model

We will use the same Conformer Transducer model used in the `Buffered Transducer Inference` tutorial, which will provide a fair comparison between the proposed merge algorithms described here.

In [ ]:
import torch
import nemo.collections.asr as nemo_asr
import contextlib
import gc

# Helper for torch amp autocast
if torch.cuda.is_available():
    autocast = torch.cuda.amp.autocast
else:
    @contextlib.contextmanager
    def autocast():
        print("AMP was not available, using FP32!")
        yield

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
pretrained_model_name = "stt_en_conformer_transducer_large"

In [ ]:
# Clear up memory
torch.cuda.empty_cache()
gc.collect()
model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained(pretrained_model_name, map_location=device)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'  # You can transcribe even longer samples on the CPU, though it will take much longer !
model = model.to(device)
model.freeze()

# Longest Common Subsequence Merge

Below, we construct the `Longest Common Subsequence Merge` algorithm to merge two consecutive transcript buffers - termed `i-1`th and `i`th buffers. The concept is similar to the work discussed in the paper [Partially Overlapped Inference for Long-Form Speech Recognition](https://ieeexplore.ieee.org/document/9414941) but operates on the notion of subword buffers rather than character tokens.

In contrast to the `Middle Token` algorithm, which utilizes certain seconds of both past and future context in order to determine the "middle tokens" for that current buffer, the `LCS Merge` algorithm merges only consecutive buffers by selecting the overlap between the end of the `i-1`th buffer and the beginning of the `i`th buffer sub-word tokens, then removing the overlapped tokens from the `i`th buffer.

While the idea is simple, since the same text can be represented by a different combination of sub-words, some additional expansion steps must be accounted for to account for imperfect alignment between two buffers.

In [ ]:
### Utility Functions ###
def print_alignment(alignment):
    """
    Print an alignment matrix of the shape (m + 1, n + 1)

    Args:
        alignment: An integer alignment matrix of shape (m + 1, n + 1)
    """
    m = len(alignment)
    if m > 0:
        n = len(alignment[0])
        for i in range(m):
            for j in range(n):
                if j == 0:
                    print(f"{i:4d} |", end=" ")
                print(f"{alignment[i][j]}", end=" ")
            print()


def write_lcs_alignment_to_pickle(alignment, filepath, extras=None):
    """
    Writes out the LCS alignment to a file, along with any extras provided.

    Args:
        alignment: An alignment matrix of shape [m + 1, n + 1]
        filepath: str filepath
        extras: Optional dictionary of items to preserve.
    """
    if extras is None:
        extras = {}

    extras['alignment'] = alignment
    torch.save(extras, filepath)

## Algorithm overview

At a high level, the algorithm can be decomposed into the following steps - 

1. Construct Longest Common Subsequence suffix matrix for initial alignment of text from old and new buffers.
2. Detect partial alignment or complete alignment.
3. If partial alignment, Loop over LCS suffix matrix to perform a dual objective search: <br>
  3.1 Find largest score for LCS at the leftmost target axis: <br>
      
      This fixed cases where intermediate tokens are an exact match to subsequent buffer getting deleted (causing a larger merge than expected and incurring a high deletion rate). <br>

  3.2 Then perform greedy expansion of this suffix: <br>

      Perform a loop descending towards the end of the suffix matrix, expanding the search space. <br>
      Limit expansion width such that only one additional token can escape per step (prevent extremely far away tokens from the current position from being used for expanded) <br>
5. Perform a backward trace of the LCS suffix matrix to find detached sections to know the beginning index of slice and length of slice.
6. Finally, check that beginning index of slice < max number of buffer chunks; if true, then slice off new buffer
 

In [ ]:
# Minimum number of tokens required to assign a LCS merge step, otherwise ignore and
# select all i-1 and ith buffer tokens to merge.
MIN_MERGE_SUBSEQUENCE_LEN = 1

### LCS algorithm ###
def longest_common_subsequence_merge(X, Y, filepath=None):
    """
    Longest Common Subsequence merge algorithm for aligning two consecutive buffers.

    Base alignment construction algorithm is Longest Common Subsequence (referred to as LCS hear after)

    LCS Merge algorithm looks at two chunks i-1 and i, determines the aligned overlap at the
    end of i-1 and beginning of ith chunk, and then clips the subsegment of the ith chunk.

    Assumption is that the two chunks are consecutive chunks, and there exists at least small overlap acoustically.

    It is a sub-word token merge algorithm, operating on the abstract notion of integer ids representing the subword ids.
    It is independent of text or character encoding.

    Since the algorithm is merge based, and depends on consecutive buffers, the very first buffer is processed using
    the "middle tokens" algorithm.

    It requires a delay of some number of tokens such that:
        lcs_delay = math.floor(((total_buffer_in_secs - chunk_len_in_sec)) / model_stride_in_secs)

    Total cost of the model is O(m_{i-1} * n_{i}) where (m, n) represents the number of subword ids of the buffer.

    Args:
        X: The subset of the previous chunk i-1, sliced such X = X[-(lcs_delay * max_steps_per_timestep):]
            Therefore there can be at most lcs_delay * max_steps_per_timestep symbols for X, preserving computation.
        Y: The entire current chunk i.
        filepath: Optional filepath to save the LCS alignment matrix for later introspection.

    Returns:
        A tuple containing -
            - i: Start index of alignment along the i-1 chunk.
            - j: Start index of alignment along the ith chunk.
            - slice_len: number of tokens to slice off from the ith chunk.
        The LCS alignment matrix itself (shape m + 1, n + 1)
    """
    # LCSuff is the table with zero
    # value initially in each cell
    m = len(X)
    n = len(Y)
    LCSuff = [[0 for k in range(n + 1)] for l in range(m + 1)]

    # To store the length of
    # longest common substring
    result = 0
    result_idx = [0, 0, 0]  # Contains (i, j, slice_len)

    # Following steps to build
    # LCSuff[m+1][n+1] in bottom up fashion
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                LCSuff[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                LCSuff[i][j] = LCSuff[i - 1][j - 1] + 1

                if result <= LCSuff[i][j]:
                    result = LCSuff[i][j]
                    result_idx = [i, j, result]

            else:
                LCSuff[i][j] = 0

    # Check if perfect alignment was found or not
    # Perfect alignment is found if :
    # Longest common subsequence extends to the final row of of the old buffer
    # This means that there exists a diagonal LCS backtracking to the beginning of the new buffer
    i, j = result_idx[0:2]
    is_complete_merge = i == m

    # Perfect alignment was found, slice eagerly
    if is_complete_merge:
        length = result_idx[-1]

        # In case the LCS was incomplete - missing a few tokens at the beginning
        # Perform backtrack to find the origin point of the slice (j) and how many tokens should be sliced
        while length >= 0 and i > 0 and j > 0:
            # Alignment exists at the required diagonal
            if LCSuff[i - 1][j - 1] > 0:
                length -= 1
                i, j = i - 1, j - 1

            else:
                # End of longest alignment
                i, j, length = i - 1, j - 1, length - 1
                break

    else:
        # Expand hypothesis to catch partial mismatch

        # There are 3 steps for partial mismatch in alignment
        # 1) Backward search for leftmost LCS
        # 2) Greedy expansion of leftmost LCS to the right
        # 3) Backtrack final leftmost expanded LCS to find origin point of slice

        # (1) Backward search for Leftmost LCS
        # This is required for cases where multiple common subsequences exist
        # We only need to select the leftmost one - since that corresponds
        # to the last potential subsequence that matched with the new buffer.
        # If we just chose the LCS (and not the leftmost LCS), then we can potentially
        # slice off major sections of text which are repeated between two overlapping buffers.

        # backward linear search for leftmost j with longest subsequence
        max_j = 0
        max_j_idx = n

        i_partial = m  # Starting index of i for partial merge
        j_partial = -1  # Index holder of j for partial merge
        j_skip = 0  # Number of tokens that were skipped along the diagonal
        slice_count = 0  # Number of tokens that should be sliced

        # Select leftmost LCS
        for i_idx in range(m, -1, -1):  # start from last timestep of old buffer
            for j_idx in range(0, n + 1):  # start from first token from new buffer
                # Select the longest LCSuff, while minimizing the index of j (token index for new buffer)
                if LCSuff[i_idx][j_idx] > max_j and j_idx <= max_j_idx:
                    max_j = LCSuff[i_idx][j_idx]
                    max_j_idx = j_idx

                    # Update the starting indices of the partial merge
                    i_partial = i_idx
                    j_partial = j_idx

        # EARLY EXIT (if max subsequence length <= MIN merge length)
        # Important case where there is long silence
        # The end of one buffer will have many blank tokens, the beginning of new buffer may have many blank tokens
        # As such, LCS will potentially be from the region of actual tokens.
        # This can be detected as the max length of the suffix in LCS
        # If this max length of the leftmost suffix is less than some margin, avoid slicing all together.
        if max_j <= MIN_MERGE_SUBSEQUENCE_LEN:
            # If the number of partiial tokens to be deleted are less than the minimum,
            # dont delete any tokens at all.

            i = i_partial
            j = 0
            result_idx[-1] = 0

        else:
            # Some valid long partial alignment was found
            # (2) Expand this alignment along the diagonal *downwards* towards the end of the old buffer
            # such that i_partial = m + 1.
            # This is a common case where due to LSTM state or reduced buffer size, the alignment breaks
            # in the middle but there are common subsequences between old and new buffers towards the end
            # We can expand the current leftmost LCS in a diagonal manner downwards to include such potential
            # merge regions.

            # Expand current partial subsequence with co-located tokens
            i_temp = i_partial + 1  # diagonal next i
            j_temp = j_partial + 1  # diagonal next j

            j_exp = 0  # number of tokens to expand along the diagonal
            j_skip = 0  # how many diagonals didn't have the token. Incremented by 1 for every row i

            for i_idx in range(i_temp, m + 1):  # walk from i_partial + 1 => m + 1
                j_any_skip = 0  # If the diagonal element at this location is not found, set to 1
                # j_any_skip expands the search space one place to the right
                # This allows 1 diagonal misalignment per timestep i (and expands the search for the next timestep)

                # walk along the diagonal corresponding to i_idx, plus allowing diagonal skips to occur
                # diagonal elements may not be aligned due to ASR model predicting
                # incorrect token in between correct tokens
                for j_idx in range(j_temp, j_temp + j_skip + 1):
                    if j_idx < n + 1:
                        if LCSuff[i_idx][j_idx] == 0:
                            j_any_skip = 1
                        else:
                            j_exp = 1 + j_skip + j_any_skip

                # If the diagonal element existed, dont expand the search space,
                # otherwise expand the search space 1 token to the right
                j_skip += j_any_skip

                # Move one step to the right for the next diagonal j corresponding to i
                j_temp += 1

            # reset j_skip, augment j_partial with expansions
            j_skip = 0
            j_partial += j_exp

            # (3) Given new leftmost j_partial with expansions, backtrack the partial alignments
            # counting how many diagonal skips occurred to compute slice length
            # as well as starting point of slice.

            # Partial backward trace to find start of slice
            while i_partial > 0 and j_partial > 0:
                if LCSuff[i_partial][j_partial] == 0:
                    # diagonal skip occurred, move j to left 1 extra time
                    j_partial -= 1
                    j_skip += 1

                if j_partial > 0:
                    # If there are more steps to be taken to the left, slice off the current j
                    # Then loop for next (i, j) diagonal to the upper left
                    slice_count += 1
                    i_partial -= 1
                    j_partial -= 1

            # Recompute total slice length as slice count along diagonal
            # plus the number of diagonal skips
            i = max(0, i_partial)
            j = max(0, j_partial)
            result_idx[-1] = slice_count + j_skip

    # Set the value of i and j
    result_idx[0] = i
    result_idx[1] = j

    if filepath is not None:
        extras = {
            "is_complete_merge": is_complete_merge,
            "X": X,
            "Y": Y,
            "slice_idx": result_idx,
        }
        write_lcs_alignment_to_pickle(LCSuff, filepath=filepath, extras=extras)
        print("Wrote alignment to :", filepath)

    return result_idx, LCSuff




## Merge Overview

Now that the LCS suffix matrix has been backtracked to obtain the starting index of the slice and the length of the slice, the merge algorithm has a simple task of simply slicing off a portion of the new buffer.

There are a few cases when merging buffers, discussed below : 

1. If there is no future context provided (i.e., chunk size == buffer size), then there is no need to merge tokens since there is no overlap possible. Then, concatenate all tokens emitted from the previous and current buffers.

2. For the first buffer, since there isn't a "previous buffer", utilize the `Middle Token` algorithm to select tokens of the first buffer and do not perform LCS merge.

3. To reduce the quadratic cost of the LCS suffix matrix computation, we pre-slice a subset of the tokens by the limit of `lcs_delay * max_symbols_per_step`. 

  > Since $lcs\_delay = \frac{total\_buffer\_in\_seconds \: - \: chunk\_size\_in\_sec}{model\_stride\_in\_sec}$, it is usually just a small amount such as 10-20 time steps. Overall, this limits the number of tokens in the previous buffer to close to 100 or so tokens, thereby limiting the cost of the LCS suffix matrix.

4. Compute the start index and slice length using the computed LCS merge.

5. Slice off the new data (`i`th chunk)

6. Merge the previous and current subset of the chunk and return the merged buffer.

In [ ]:
def lcs_alignment_merge_buffer(buffer, data, delay, model, max_steps_per_timestep: int = 5, filepath: str = None):
    """
    Merges the new text from the current frame with the previous text contained in the buffer.

    The alignment is based on a Longest Common Subsequence algorithm, with some additional heuristics leveraging
    the notion that the chunk size is >= the context window. In case this assumptio is violated, the results of the merge
    will be incorrect (or at least obtain worse WER overall).
    """
    # If delay timesteps is 0, that means no future context was used. Simply concatenate the buffer with new data.
    if delay < 1:
        buffer += data
        return buffer

    # If buffer is empty, simply concatenate the buffer and data.
    if len(buffer) == 0:
        buffer += data
        return buffer

    # Prepare a subset of the buffer that will be LCS Merged with new data
    search_size = int(delay * max_steps_per_timestep)
    buffer_slice = buffer[-search_size:]

    # Perform LCS Merge
    lcs_idx, lcs_alignment = longest_common_subsequence_merge(buffer_slice, data, filepath=filepath)

    # Slice off new data
    # i, j, slice_len = lcs_idx
    slice_idx = lcs_idx[1] + lcs_idx[-1]  # slice = j + slice_len
    data = data[slice_idx:]

    # Concat data to buffer
    buffer += data
    return buffer

# LCS Merge algorithm as a basis for Buffered ASR

Next, let us extend the previous `BatchedFrameASRRNNT` codebase for Buffered Transducer to incorporate the new merge algorithm.

We will note that the vast majority of the code remains unchanged - only the `transcribe` function has been changed to utilize the new merge algorithm.

In [ ]:
from nemo.collections.asr.parts.utils import streaming_utils
from torch.utils.data import DataLoader


class LongestCommonSubsequenceBatchedFrameASRRNNT(streaming_utils.BatchedFrameASRRNNT):
    """
    Implements a token alignment algorithm for text alignment instead of middle token alignment.

    For more detail, read the docstring of longest_common_subsequence_merge().
    """

    def __init__(self, asr_model, frame_len=1.6, total_buffer=4.0,
        batch_size=4, max_steps_per_timestep: int = 5, stateful_decoding: bool = False, 
        alignment_basepath: str = None,
    ):
        '''
        Args:
            asr_model: An RNNT model.
            frame_len: frame's duration, seconds.
            total_buffer: duration of total audio chunk size, in seconds.
            batch_size: Number of independent audio samples to process at each step.
            max_steps_per_timestep: Maximum number of tokens (u) to process per acoustic timestep (t).
            stateful_decoding: Boolean whether to enable stateful decoding for preservation of state across buffers.
            alignment_basepath: Str path to a directory where alignments from LCS will be preserved for later analysis.
        '''
        super().__init__(asr_model, frame_len, total_buffer, batch_size, max_steps_per_timestep, stateful_decoding)
        self.sample_offset = 0
        self.lcs_delay = -1
        self.alignment_basepath = alignment_basepath

    def transcribe(
        self, tokens_per_chunk: int, delay: int,
    ):
        if self.lcs_delay < 0:
            raise ValueError(
                "Please set LCS Delay values as `(buffer_duration - chunk_duration) / model_stride_in_secs`"
            )

        self.infer_logits()

        self.unmerged = [[] for _ in range(self.batch_size)]
        for idx, alignments in enumerate(self.all_alignments):

            signal_end_idx = self.frame_bufferer.signal_end_index[idx]
            if signal_end_idx is None:
                raise ValueError("Signal did not end")

            for a_idx, alignment in enumerate(alignments):

                # Middle token algorithm for the first chunk
                if a_idx == 0:
                    alignment = alignment[len(alignment) - 1 - delay :]
                    ids, toks = self._alignment_decoder(alignment, self.asr_model.tokenizer, self.blank_id)

                    if len(ids) > 0:
                        self.unmerged[idx] = streaming_utils.inplace_buffer_merge(
                            self.unmerged[idx], ids, delay, model=self.asr_model,
                        )

                else:
                    # Use LCS Merge algorithm for remaining chunks
                    ids, toks = self._alignment_decoder(alignment, self.asr_model.tokenizer, self.blank_id)
                    if len(ids) > 0 and a_idx < signal_end_idx:
                        
                        # Preserve the LCS alignments if a alignment path was provided
                        if self.alignment_basepath is not None:
                            basepath = self.alignment_basepath
                            sample_offset = self.sample_offset + idx
                            alignment_offset = a_idx
                            path = os.path.join(basepath, str(sample_offset))

                            os.makedirs(path, exist_ok=True)
                            path = os.path.join(path, "alignment_" + str(alignment_offset) + '.pt')

                            filepath = path
                        else:
                            filepath = None 

                        # Perform LCS merge
                        self.unmerged[idx] = lcs_alignment_merge_buffer(
                            self.unmerged[idx],
                            ids,
                            self.lcs_delay,
                            model=self.asr_model,
                            max_steps_per_timestep=self.max_steps_per_timestep,
                            filepath=filepath,
                        )

        output = []
        for idx in range(self.batch_size):
            output.append(self.greedy_merge(self.unmerged[idx]))
        return output


# Comparing "Middle Token" and "LCS Merge"

While we propose the two algorithms - `Middle Token` and `LCS Merge`, we would recommend using either algorithm in the appropriate circumstances. The `Middle Token` algorithm performs well in general, and its mistakes are often fewer than the `LCS Merge` algorithm but requires future context, which may increase latency by a small amount. There are also cases where `LCS Merge` may select better alignments and result in slightly better scores for some audio samples.

In general, we propose these approaches to discuss further and research merge algorithms that show some trade-off between latency and accuracy.

In [ ]:
#@title Change Decoding Strategy for Buffered Inference
# Change Decoding Config
from omegaconf import OmegaConf, open_dict

decoding_cfg = model.cfg.decoding
with open_dict(decoding_cfg):
    decoding_cfg.strategy = "greedy_batch"
    decoding_cfg.preserve_alignments = True  # required to compute the middle token for transducers.
    decoding_cfg.fused_batch_size = -1  # temporarily stop fused batch during inference.

model.change_decoding_strategy(decoding_cfg)

In [ ]:
#@title Helper methods to transcribe audio in buffered mode

import tqdm
import math

def transcribe_buffers(asr_decoder, audio_filepaths, chunk_len_in_secs, buffer_len_in_secs, model_stride):
  model.freeze()
  model_stride_in_secs = asr_decoder.asr_model.cfg.preprocessor.window_stride * model_stride
  tokens_per_chunk = math.ceil(chunk_len_in_secs / model_stride_in_secs)
  mid_delay = math.ceil((chunk_len_in_secs + (buffer_len_in_secs - chunk_len_in_secs) / 2) / model_stride_in_secs)
  lcs_delay = math.floor(((buffer_len_in_secs - chunk_len_in_secs)) / model_stride_in_secs)

  hyps = []
  batch_size = asr_decoder.batch_size 

  with torch.inference_mode():
    with torch.cuda.amp.autocast():
      batch = []
      asr_decoder.sample_offset = 0
      asr_decoder.lcs_delay = lcs_delay

      for idx in tqdm.tqdm(range(len(audio_filepaths)), desc='Sample:', total=len(audio_filepaths)):
          batch.append(audio_filepaths[idx],)

          if len(batch) == batch_size:
              audio_files = [sample for sample in batch]

              asr_decoder.reset()
              asr_decoder.read_audio_file(audio_files, mid_delay, model_stride_in_secs)
              hyp_list = asr_decoder.transcribe(tokens_per_chunk, mid_delay)
              hyps.extend(hyp_list)

              batch.clear()
              asr_decoder.sample_offset += batch_size

      if len(batch) > 0:
          asr_decoder.batch_size = len(batch)
          asr_decoder.frame_bufferer.batch_size = len(batch)
          asr_decoder.reset()

          audio_files = [sample for sample in batch]
          asr_decoder.read_audio_file(audio_files, mid_delay, model_stride_in_secs)
          hyp_list = asr_decoder.transcribe(tokens_per_chunk, mid_delay)
          hyps.extend(hyp_list)

          batch.clear()
          asr_decoder.sample_offset += len(batch)
  
  print("Finished transcribing audio files")
  return hyps

## Select data subset

On the GPU, it would take a few minutes to perform inference for the entire dataset, but on the CPU, it would take quite a long time. While the defaults will exist for the whole dataset, if only the CPU is available for some reason, we encourage you to subsample the dataset.

In [ ]:
#@title Manifest helper
import json
import numpy as np
from nemo.collections.asr.parts.utils.manifest_utils import read_manifest


def subset_manifest(manifest, num_samples):
  rng = np.random.RandomState(seed=0)
  num_samples = min(len(manifest), num_samples)

  if num_samples < len(manifest):
    ids = rng.choice(np.arange(len(manifest)), size=num_samples, replace=False)
    ids = ids.tolist()
  else:
    ids = np.arange(len(manifest)).tolist()

  sub_manifest = []
  for idx in ids:
    sub_manifest.append(manifest[idx])
  
  print(f"Prepared subset manifest with {len(sub_manifest)} samples.")
  return sub_manifest

In [ ]:
manifest_data = read_manifest(manifest)
print(f"Read {len(manifest_data)} samples from manifest {manifest}")

In [ ]:
num_samples = len(manifest_data)

#########################################################
sub_manifest = subset_manifest(manifest_data, num_samples)
audio_filepaths = [sample['audio_filepath'] for sample in sub_manifest]
ground_texts = [sample['text'] for sample in sub_manifest]

## Buffered Inference arguments

Below we detail some critical arguments for buffered transducer inference. Note that the primary difference between streaming and buffered inference would be the chunk length, with larger values contributing to a lower word error rate but higher latency. 

In [ ]:
chunk_len_in_secs: float = 8.0
context_len_in_secs: float = 1.0
model_stride: int = 4  # 4 for conformers, 8 for contextnet

batch_size: int = 64  # Select a low value for CPU such as 4 or 8.
lcs_alignments_path = os.path.join(os.getcwd(), "lcs_alignments")
max_steps_per_timestep: int = model.cfg.decoding.greedy.max_symbols
        
##########################################################################
buffer_len_in_secs = chunk_len_in_secs + 2* context_len_in_secs


## Baseline: Middle Token Predictions

Now compute the transcriptions over the data subset using the baseline algorithm - `Middle Token`. 

In [ ]:
asr_middle = streaming_utils.BatchedFrameASRRNNT(model, chunk_len_in_secs, buffer_len_in_secs,
                                                 batch_size=batch_size, max_steps_per_timestep=max_steps_per_timestep)

In [ ]:
middle_transcripts = transcribe_buffers(asr_middle, audio_filepaths, chunk_len_in_secs, buffer_len_in_secs, model_stride)

In [ ]:
from nemo.collections.asr.metrics.wer import word_error_rate

wer_middle = word_error_rate(middle_transcripts, ground_texts, use_cer=False)
print("Middle token algorithm WER :", wer_middle)

## LCS Merge Predictions

Next, let us compute the transcriptions over the data subset using the `LCS Merge` algorithm.

In [ ]:
asr_lcs = LongestCommonSubsequenceBatchedFrameASRRNNT(model, chunk_len_in_secs, buffer_len_in_secs,
                                                      batch_size=batch_size, max_steps_per_timestep=max_steps_per_timestep,
                                                      alignment_basepath=lcs_alignments_path)

In [ ]:
lcs_transcripts = transcribe_buffers(asr_lcs, audio_filepaths, chunk_len_in_secs, buffer_len_in_secs, model_stride)

In [ ]:
wer_lcs = word_error_rate(lcs_transcripts, ground_texts, use_cer=False)
print("LCS algorithm WER :", wer_lcs)

# Compare the text predictions from the two merge algorithms

Depending on the data subset chosen (or randomly sampled), the WER for this algorithm may be higher or lower than the baseline. Note that if you select all the samples in the dataset, then the WER of this method is slightly higher than the baseline.

We will do a more in-depth analysis of the failure cases below.

In [ ]:
def compare_algorithms(ground_truth, middle_transcripts, lcs_transcripts, use_cer=False):
  worse = []
  better = []
  same = []

  for idx, (ground_text, middle_data, lcs_data) in enumerate(zip(ground_truth, middle_transcripts, lcs_transcripts)):
      middle_wer = word_error_rate([middle_data], [ground_text], use_cer=use_cer)
      lcs_wer = word_error_rate([lcs_data], [ground_text], use_cer=use_cer)

      if middle_wer < lcs_wer:
          worse.append((idx, ground_text, middle_data, lcs_data))

      elif middle_wer > lcs_wer:
          better.append((idx, ground_text, middle_data, lcs_data))
      
      else:
          same.append((idx, ground_text, middle_data, lcs_data))
  
  print("Number of samples where both algorithms obtained same WER :", len(same))
  print("Number of samples LCS merge was better than middle ground :", len(better))
  print("Number of samples LCS merge was worse than middle ground  :", len(worse))
  return same, better, worse

In [ ]:
both_same, lcs_better, lcs_worse = compare_algorithms(ground_texts, middle_transcripts, lcs_transcripts, use_cer=False)

# EXTRA: Compare the alignment matrices of LCS

Over the entire dataset, there would be some samples where the `LCS Merge` algorithm did better than the `Middle Token` algorithm and vice-versa. Below, we will take a sample-level look at such cases, and since the `LCS Merge` algorithm is an alignment-based technique, we can visualize the alignment itself and determine what cases it failed and the source of the error in the alignment itself.


In [ ]:
#@title LCS Alignment helper functions

from torch.cuda import stream
import torch
import glob

def try_load_alignments(alignmen_dir, idx):
    basepath = os.path.abspath(alignmen_dir)
    sample_path = os.path.join(basepath, str(idx))
    
    files = list(glob.glob(f"{sample_path}/*.pt"))
    alignments = []
    if len(files) > 0:
        print(f"Found {len(files)} alignments")
        for i in range(1, len(files) + 1):
            path = os.path.join(sample_path, f"alignment_{i}.pt")
            alignments.append(torch.load(path))
    
    return alignments

def extract_alignments(alignments):
    all_alignments = []
    for data in alignments:
        all_alignments.append(data['alignment'])
    
    return all_alignments

def find_first_sample_with_alignment(alignment_dir, samples, start_idx: int = 0):
    for idx in range(start_idx, len(samples)):
        sample = samples[idx]
        alignments = try_load_alignments(alignment_dir, sample[0])
        if len(alignments) > 0:
            return idx

def print_alignment(alignment):
    m = len(alignment)
    if m > 0:
        streaming_utils.print_alignment(alignment)

def greedy_decode(asr_model, preds):
    decoded_prediction = [p for p in preds]
    hypothesis = asr_model.tokenizer.ids_to_text(decoded_prediction)
    hyp_subwords = asr_model.tokenizer.ids_to_tokens(decoded_prediction)
    return hypothesis, hyp_subwords

def display_alignment_merge(alignment_path, sample, 
                            print_xy_token_ids: bool = False,
                            print_xy_text: bool = True,
                            print_alignments: bool = True,
                            max_steps_per_timestep: int = None):

  model_stride_in_secs = model.cfg.preprocessor.window_stride * model_stride
  lcs_delay = math.floor(((buffer_len_in_secs - chunk_len_in_secs)) / model_stride_in_secs)
  if max_steps_per_timestep is None:
    max_steps_per_timestep = model.cfg.decoding.greedy.max_symbols

  alignments_meta = try_load_alignments(alignment_path, sample[0])
  alignments = extract_alignments(alignments_meta)

  print("Sample Meta Info")
  print()
  if print_xy_token_ids:
    for idx in range(len(alignments_meta)):
      print("X", alignments_meta[idx]['X'])
      print("Y", alignments_meta[idx]['Y'])
      print()
    print()

  if print_xy_text:
    for idx in range(len(alignments_meta)):
      x_text, x_subwords = greedy_decode(model, alignments_meta[idx]['X'])
      y_text, y_subwords = greedy_decode(model, alignments_meta[idx]['Y'])
      
      X = alignments_meta[idx]['X'].copy()
      Y = alignments_meta[idx]['Y'].copy()
      search_size = int(lcs_delay * max_steps_per_timestep)

      result = streaming_utils.lcs_alignment_merge_buffer(X, Y, lcs_delay, model, max_steps_per_timestep)
      result, _ = greedy_decode(model, result)

      print("Alignment step :", idx)
      print("Is there perfect alignment match for merge :", alignments_meta[idx]['is_complete_merge']) 
      print()
      print("Ground truth :", sample[1])
      print("Merge Result :", result)
      print("X text       :", x_text)
      print("Y text       :", y_text)
      print("Slice index  :", alignments_meta[idx]['slice_idx'][1:], "(start index, slice length in subword tokens)")
      print()

      if print_alignments:
        print_alignment(alignments[idx][-search_size:])
        print()
      print()
    print()

        

## Worse alignment

Let us search for a sample where the `LCS Merge` did worse than the `Middle Token` algorithm. 

Such cases are necessary to analyze because it is visually apparent where the alignment went wrong. We can determine if there could be an extension to this algorithm to further improve such cases.


In [ ]:
worse_idx = find_first_sample_with_alignment(lcs_alignments_path, lcs_worse, start_idx=0)
worse_sample = lcs_worse[worse_idx]

print("A sample where LCS did worse than Middle Token merge algorithm :")
print("The texts are structured as (Ground Truth, Middle Token, LCS Merge)")
worse_sample

In [ ]:
display_alignment_merge(lcs_alignments_path, worse_sample, print_xy_token_ids=False)

## Better alignment

Next, let us search for a sample where the `LCS Merge` did better than the `Middle Token` algorithm. 

Such cases are also essential to analyze because it is visually apparent where the alignment was better. We can determine if we can improve the `Middle Token` algorithm.

In [ ]:
better_idx = find_first_sample_with_alignment(lcs_alignments_path, lcs_better, start_idx=0)
better_sample = lcs_better[better_idx]

print("A sample where LCS did better than Middle Token merge algorithm :")
print("The texts are structured as (Ground Truth, Middle Token, LCS Merge)")
better_sample

In [ ]:
display_alignment_merge(lcs_alignments_path, better_sample)

# Final notes

Following the [Buffered Transducer Inference](https://github.com/NVIDIA/NeMo/blob/stable/tutorials/asr/Buffered_Transducer_Inference.ipynb) tutorial and designing a token merge algorithm that can be a simple extension to the baseline `Middle Token` algorithm, we see that there are cases where both algorithms have their uses. 

To expand our research effort on developing more sophisticated streaming / buffered transducer inference methods, we encourage the users to try these algorithms in script format for efficient inference on large datasets - available at [ASR Chunked Streaming Inference](https://github.com/NVIDIA/NeMo/blob/stable/examples/asr/asr_chunked_inference/rnnt/speech_to_text_buffered_infer_rnnt.py).
